In [ ]:
import 

# Static Model

In [ ]:
# Initialize random seed
seed = 30026
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# Define data of the problem
num_classes = 15
num_timesteps = 5

# Decide whever to be verbose with display outputs and set device
verbose = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define hyperparameters
with open('hyperparameters.json', 'r') as file:
    params = json.load(file)

static_params = params["static_model"]
dynamic_params = params["dynamic_model"]

# Load file
data = pd.read_csv(params["file_path"])

In [ ]:
# Split dataset in train and test sets keeping balance for the static label localization
data_train, data_test = train_test_split(data, test_size=params["test_size"], random_state=seed, stratify = data['static_localization'])

# Create dataloaders
dataloader_train, dataloader_test = create_data_loaders(data_train, data_test,params["batch_size"])